_First code cell from custom template `dsml.ipynb` in [user_path]\Anaconda3\envs\dsml\Lib\site-packages\jupyterlab_templates\templates\jupyterlab_templates_. <br>
_See Tim Paine's [`jupyter_lab templates` extention](https://github.com/timkpaine/jupyterlab_templates)._

In [4]:
import sys
from pathlib import Path, PurePath as PPath

print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))

def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    if up:
        # NB: Path does not have a str method.
        newp = str(PPath(this_path).parent)
    else:
        newp = str(PPath(this_path)) 
    
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


def get_project_dirs(which=['data', 'images'], nb_folder='notebooks'):
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder):
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
import numpy as np
import scipy as sp
from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp

# Filtered dir() for method discovery:
def filter_module_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work together
              to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]

def get_mdl_pkgs(alib):
    import inspect
    "Inspect module hierarchy on two levels ony."
    for name, mdl in inspect.getmembers(alib, inspect.ismodule):
        print('\n{:>13} : {}'.format(mdl.__name__, filter_dir(mdl)))
        for mdl_name, mdl_sub in inspect.getmembers(mdl, inspect.ismodule):
            if mdl_sub.__doc__:
                print('\n{:>20} : {}'.format(mdl_name, mdl_sub.__doc__.strip()))
                

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import HTML, Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    from IPython import get_ipython
    from IPython.display import HTML, Markdown
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """
    accepted = ['info', 'warning', 'danger']
    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)

# autoreload extension
from IPython import get_ipython
ipython = get_ipython()

if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

Python ver: 3.6.7 (default, Feb 28 2019, 07:28:18) [MSC v.1900 64 bit (AMD64)]
Python env: dsml
Currrent dir: C:\Users\catch\Documents\GitHub\AI\Exercises\1_Constraint Satisfaction

Path added to sys.path: C:\Users\catch\Documents\GitHub\AI\Exercises\1_Constraint Satisfaction


<div class="alert alert-info" ><h1>Work In Progress</h1></div>

---
---
# A class for solving crypto arithmetic (CA) puzzles using Z3.

This project emanates from the CA exercise in `AIND-Constraint_Satisfaction.ipynb` and the CA challenges:
>### Cryptarithmetic Challenges
>0. Search online for [more cryptarithmetic puzzles](https://www.reddit.com/r/dailyprogrammer/comments/7p5p2o/20180108_challenge_346_easy_cryptarithmetic_solver/) (or create your own). Come to office hours or join a discussion channel to chat with your peers about the trade-offs between monolithic constraints & splitting up the constraints. (Is one way or another easier to generalize or scale with new problems? Is one of them faster for large or small problems?)
>0. Can you extend the solution to handle complex puzzles (e.g., using multiplication WORD1 x WORD2 = OUTPUT)?
---

# [Reddit] Challenge #346 [Easy] Cryptarithmetic Solver 

## Challenge Input:    
```
"WHAT + WAS + THY == CAUSE"

"HIS + HORSE + IS == SLAIN"

"HERE + SHE == COMES"

"FOR + LACK + OF == TREAD"

"I + WILL + PAY + THE == THEFT"
```

## Bonus:
```
"TEN + HERONS + REST + NEAR + NORTH + SEA + SHORE + AS + TAN + TERNS + SOAR + TO + ENTER + THERE + AS + HERONS + NEST + ON + STONES + AT + SHORE + THREE + STARS + ARE + SEEN + TERN + SNORES + ARE + NEAR == SEVVOTH"

"SO + MANY + MORE + MEN + SEEM + TO + SAY + THAT + THEY + MAY + SOON + TRY + TO + STAY + AT + HOME +  SO + AS + TO + SEE + OR + HEAR + THE + SAME + ONE + MAN + TRY + TO + MEET + THE + TEAM + ON + THE + MOON + AS + HE + HAS + AT + THE + OTHER + TEN == TESTS"

"THIS + A + FIRE + THEREFORE + FOR + ALL + HISTORIES + I + TELL + A + TALE + THAT + FALSIFIES + ITS + TITLE + TIS + A + LIE + THE + TALE + OF + THE + LAST + FIRE + HORSES + LATE + AFTER + THE + FIRST + FATHERS + FORESEE + THE + HORRORS + THE + LAST + FREE + TROLL + TERRIFIES + THE + HORSES + OF + FIRE + THE + TROLL + RESTS + AT + THE + HOLE + OF + LOSSES + IT + IS + THERE + THAT + SHE + STORES + ROLES + OF + LEATHERS + AFTER + SHE + SATISFIES + HER + HATE + OFF + THOSE + FEARS + A + TASTE + RISES + AS + SHE + HEARS + THE + LEAST + FAR + HORSE + THOSE + FAST + HORSES + THAT + FIRST + HEAR + THE + TROLL + FLEE + OFF + TO + THE + FOREST + THE + HORSES + THAT + ALERTS + RAISE + THE + STARES + OF + THE + OTHERS + AS + THE + TROLL + ASSAILS + AT + THE + TOTAL + SHIFT + HER + TEETH + TEAR + HOOF + OFF + TORSO + AS + THE + LAST + HORSE + FORFEITS + ITS + LIFE + THE + FIRST + FATHERS + HEAR + OF + THE + HORRORS + THEIR + FEARS + THAT + THE + FIRES + FOR + THEIR + FEASTS + ARREST + AS + THE + FIRST + FATHERS + RESETTLE + THE + LAST + OF + THE + FIRE + HORSES + THE + LAST + TROLL + HARASSES + THE + FOREST + HEART + FREE + AT + LAST + OF + THE + LAST + TROLL + ALL + OFFER + THEIR + FIRE + HEAT + TO + THE + ASSISTERS + FAR + OFF + THE + TROLL + FASTS + ITS + LIFE + SHORTER + AS + STARS + RISE + THE + HORSES + REST + SAFE + AFTER + ALL + SHARE + HOT + FISH + AS + THEIR + AFFILIATES + TAILOR + A + ROOFS + FOR + THEIR + SAFE == FORTRESSES"
```

---
### Notes from a pythonista:  
<strong> I import z3 the pythonic way:</strong> 
```import z3``` 
instead of  
```from z3 import *``` as seen in numerous tutorials.  

<strong> Why?</strong>
<strong>
1.  It is not recommended as this can corrupt the namespace (some functions from different libraries could have the same name).
2.  I usually inspect a module **in order to learn** about its various functions and methods, so I used the root object for that (anlong with my function `filter_module_dir`).

The slight drawback &mdash; apart from from having to type a little more &mdash; is that some variables such as those returned by the `Solver.check()` function (i.e. `sat, unsat, unkown`), are not defined.  
The following lines use the [`CheckSatResult()` docs](https://z3prover.github.io/api/html/classz3py_1_1_check_sat_result.html) to define them:
</strong>

In [2]:
from collections import defaultdict
from itertools import product

import z3

sat = z3.z3.CheckSatResult(z3.Z3_L_TRUE)
unsat = z3.z3.CheckSatResult(z3.Z3_L_FALSE)
# else: unkown

---
## What's in Z3?

The cell below shows how I list a module's methods to learn about them:

In [5]:
z3_mdl = filter_module_dir(z3)

z3_data = defaultdict(list)
for entry in z3_mdl:
    z3_data[entry[0]].append(entry)

z3_data['a']
z3_data['A']
z3_constants = z3_data['Z']
z3_constants[:10]

['addressof', 'alignment', 'append_log', 'args2params']

['ARRAY',
 'AlgebraicNumRef',
 'And',
 'AndThen',
 'ApplyResult',
 'ApplyResultObj',
 'ArgumentError',
 'ArithRef',
 'ArithSortRef',
 'Array',
 'ArrayRef',
 'ArraySort',
 'ArraySortRef',
 'Ast',
 'AstMap',
 'AstMapObj',
 'AstRef',
 'AstVector',
 'AstVectorObj',
 'AtLeast',
 'AtMost']

['Z3Exception',
 'Z3PPObject',
 'Z3_APP_AST',
 'Z3_ARRAY_SORT',
 'Z3_BOOL_SORT',
 'Z3_BV_SORT',
 'Z3_DATATYPE_SORT',
 'Z3_DEBUG',
 'Z3_DEC_REF_ERROR',
 'Z3_EXCEPTION']

In [151]:
z3.z3.AstRef?

Init signature: z3.z3.AstRef(ast, ctx=None)
Docstring:      AST are Direct Acyclic Graphs (DAGs) used to represent sorts, declarations and expressions.
File:           c:\users\catch\anaconda3\envs\dsml\lib\site-packages\z3\z3.py
Type:           type
Subclasses:     SortRef, FuncDeclRef, ExprRef


---
---
# The CryptoArithmeticZ3Solver class definition

In [250]:
import z3

sat = z3.z3.CheckSatResult(z3.Z3_L_TRUE)
unsat = z3.z3.CheckSatResult(z3.Z3_L_FALSE)


class CryptoArithmeticZ3Solver():
    def __init__(self, crypto, solution=None, op='+'):
        msg = 'Crypto format: "WORD + IS + THIS == MUCH"\n'
        if op not in crypto:
            raise TypeError(msg+f'Missing operator {op}')
        if '==' not in crypto:
            raise TypeError(msg+f'Missing equal "=="')
            
        self.op = op.strip()   
        self.n_op = crypto.count(op)
        
        # Get list of words to appear on each line (summands if +)
        words = [w.strip() for w in crypto.split(self.op)]
        tail = words.pop(-1)
        words += [w.strip() for w in tail.split('==')]
        self.words = words
        self.result_word = self.words[-1]
        
        # Set other props with one loop:
        words_len = list()
        first_ltrs = list()
        letters = set()
        maxl = 0
        for w in words:
            wlen = len(w)
            words_len.append(wlen)
            if wlen > maxl:
                maxl = wlen
            first_ltrs.append(w[0])
            letters = set([''.join(l) for l in w])
        
        self.max_len = maxl
        self.letters = letters
        self.first_letters = first_ltrs
        
        self.initial_lines = self._get_initial_lines(crypto)
        self.solution = solution
        self.solution_lines = self._get_solution_lines()
    
    def _get_word_repr(self, word):
        """
        Return a string of letters in word separated with a space.
        Used in _get_initial_lines().
        """
        s = ''
        for l in word:
            s += f' {l}'
        return s
    
    def _get_letters_repr(self):
        """
        Return a tuple of letters separated by a space:
             (unique letters in puzzle, start_letters, other_letters).
        Used in _solver_setup().
        """
        L, F, O = '', '', ''
        for l in self.letters:
            L += f'{l} '
            if l in self.first_letters:
                F += f'{l} '
            else:
                O += f'{l} '
        return (L[:-1], F[:-1], O[:-1])
    
        
    def _get_initial_lines(self, crypto):
        """Pretty print the cryptarithmetric sting and its solution.
        Arguments:
        ----------
        (str) crypto format: "WORD + IS + THIS == MUCH"
        (dict) solution: letter as key
        """
        lines = [] # -> [('T W O ', 'TWO')]
        line = ''
        n_op = crypto.count(op)
        
        for p in range(n_op + 2):    
            if p == (n_op + 1):
                sub = '{0:{fill}>{width}}'.format('-',fill='-', width=1+self.max_len*2)
                lines.append((sub, '-'))
            
            s = self._get_word_repr(words[p])
            
            if (p > 0) and (p < (n_op + 1)):
                line = '{op} {0:>{width}}'.format(s,
                                                  op=self.op,
                                                  width=self.max_len*2-2)
            else:
                line = '{:>{width}}'.format(s, width=self.max_len*2)
            lines.append((line, words[p]))
            
        return lines
    
    
    def _get_solution_lines(self):
        if self.solution is None:
            return []
        else:
            # self.solution hold the solver model
            m = self.solution

            out = dict((str(v), m[v]) for v in m.decls())
            print(out)
            
            #print(f"  T W O  :    {out['T']} {out['W']} {out['O']}")
            #print(f"+ T W O  :  + {out['T']} {out['W']} {out['O']}")
            #print("-------  :  -------")
            #print(f"F O U R  :  {out['F']} {out['O']} {out['U']} {out['R']}")
            
            lines = [] # [('T W O ', 'TWO')]
            
            # get the initial line & exapnd them with solution
            for i, (line, word) in enumerate(self.initial_lines):
                values = ''
                for l in word:
                    values += f' {out[str(l)]}'
                    
                if (i > 0) and (i < len(self.initial_lines)-1):
                    line += ' -> ' + '{op} {0:>{width}}'.format(values,
                                                                op=self.op,
                                                                width=self.max_len*2-2)
                else:
                    line += ' -> ' + '{:>{width}}'.format(values, width=self.max_len*2)
                lines.append(line)
    
            return lines
        
        
    def display(self):
        s = ''
        lines_out = self.initial_lines if self.solution is None else self.solution_lines
        
        for L in lines_out:
            s +=f'{L[0]}\n'
        print(s)
            
        
    def _solver_setup(self):
        # create a Z3 CSP solver instance
        solver = z3.Solver()
        
        # get 'l e t t e r s':
        ltrs, first_ltrs, other_ltrs = self._get_letters_repr()

        # create variables instance for all unique letters:
        ltrs_vars = z3.Ints(ltrs)
        # constraint: all must be different
        c_distinct = [ z3.Distinct(ltrs_vars) ]
        
        # constraint: each word's 1st letter is a non-zero digit:
        c_zero_not_first = [ z3.And(1<=L, L<=9) for L in z3.Ints(first_ltrs) ]
    
        # the others can be 0
        c_digit = [ z3.And(0<=L, L<=9) for L in z3.Ints(other_ltrs) ]

        # add all input constraints to the solver:
        c_input = c_distinct + c_zero_not_first + c_digit
        solver.add(c_input)
        
        return solver

    def solve_puzzle(self):
        solver = self._solver_setup()
        if solver.check() == sat:
            self.solution = solver.model()
            self.solution_lines = self._get_solution_lines()
        else:
            print('Failed to solve.')
            self.solution = None

In [258]:
crypto_str = "THIS + IS + HIS == CLAIM"

crypto = CryptoArithmeticZ3Solver(crypto_str)
crypto.display()
crypto.letters

   T H I S
+      I S
+    H I S
-----------
 C L A I M



{'A', 'C', 'I', 'L', 'M'}

In [252]:
crypto.initial_lines

[('   T H I S', 'THIS'),
 ('+      I S', 'IS'),
 ('+    H I S', 'HIS'),
 ('-----------', '-'),
 (' C L A I M', 'CLAIM')]

In [257]:
crypto.solution.decls()

[A, M, I, C, L]

In [254]:
crypto.solve_puzzle()


{'A': 2, 'M': 3, 'I': 1, 'C': 4, 'L': 5}


KeyError: 'T'

In [255]:
crypto.solution_lines

[]

---

In [104]:
crypto_str = "THIS + IS + HIS == CLAIM"
op = '+'

In [89]:
# Set other props with one loop:
words_len = list()
first_ltrs = list()
letters = set()
maxl = 0
for w in words:
    wlen = len(w)
    words_len.append(wlen)
    if wlen > maxl:
        maxl = wlen
    first_ltrs.append(w[0])
    letters = letters.union(set([''.join(l) for l in w]))

---

In [225]:
ca_solver = ca_solver_init()
ca_solver.check()

m = ca_solver.model()
out = dict((str(v), m[v]) for v in m.decls())
out

sat

{'R': 4, 'O': 2, 'W': 3, 'U': 6, 'F': 1, 'T': 5}

In [ ]:
ca_solver_results(ca_solver)

In [146]:
def ca_solver_init():
    # create a Z3 CSP solver instance
    ca_solver = z3.Solver()
    
    ltrs = 'F O R T U W'
    # create variables instance
    ltrs_vars = z3.Ints(ltrs)
    # each F, T vars contains a decimal digit (a value in {1, ..., 9}):
    c_zero_not_first = [ z3.And(1<=D, D<=9) for D in z3.Ints('F T') ]
    # the others can be 0
    c_digit = [ z3.And(0<=D, D<=9) for D in z3.Ints('O R U W') ]
    # all must be different
    c_distinct = [ z3.Distinct(ltrs_vars) ]

    c_input = c_digit + c_zero_not_first + c_distinct
    
    # add all input constraints to the solver:
    ca_solver.add(c_input)
    return ca_solver


def ca_solver_results(s):
    if s.check() == sat:
        m = s.model()

        out = {}
        out = dict((str(v), m.evaluate(v)) for v in [F, O, R, T, U, W])

        print(f"  T W O  :    {out['T']} {out['W']} {out['O']}")
        print(f"+ T W O  :  + {out['T']} {out['W']} {out['O']}")
        print("-------  :  -------")
        print(f"F O U R  :  {out['F']} {out['O']} {out['U']} {out['R']}")
    else:
        print("failed to solve")
        
def ca_solver_results_failed(s):
    if s.check() == sat:
        m = s.model()

        out = {}
        out = dict((str(v), m.evaluate(v)) for v in m.decls()))

        print(f"  T W O  :    {out['T']} {out['W']} {out['O']}")
        print(f"+ T W O  :  + {out['T']} {out['W']} {out['O']}")
        print("-------  :  -------")
        print(f"F O U R  :  {out['F']} {out['O']} {out['U']} {out['R']}")
    else:
        print("failed to solve")

In [147]:
del F, O, R, T, U, W

In [137]:
ca_solver = z3.Solver()

F, O, R, T, U, W = z3.Ints('F O R T U W') 

# Add carry variables:
c10, c100, c1000 = z3.Ints('c10 c100 c1000')

# each F, T vars contains a decimal digit (a value in {1, ..., 9}):
c_zero_not_first = [ z3.And(1<=D, D<=9) for D in z3.Ints('F T') ]

# the others can be 0
c_digit = [ z3.And(0<=D, D<=9) for D in z3.Ints('O R U W') ]
ca_solver.add(c_zero_not_first + c_digit)

c_distinct = [ z3.Distinct([F, O, R, T, U, W]) ]
ca_solver.add(c_distinct)

ca_solver.add(2*O == R + 10 * c10)
ca_solver.add(2*W + c10 == U + 10 * c100)
ca_solver.add(2*T + c100 == O + 10 * c1000)
ca_solver.add(F == c1000)

ca_solver_results(ca_solver)

  T W O  :    7 3 4
+ T W O  :  + 7 3 4
-------  :  -------
F O U R  :  1 4 6 8


In [177]:
z3.Function?

Signature: z3.Function(name, *sig)
Docstring:
Create a new Z3 uninterpreted function with the given sorts.

>>> f = Function('f', IntSort(), IntSort())
>>> f(f(0))
f(f(0))
File:      c:\users\catch\anaconda3\envs\dsml\lib\site-packages\z3\z3.py
Type:      function


The operation of modular exponentiation calculates the remainder when an integer b (the base)  
raised to the eth power (the exponent), be, is divided by a positive integer m (the modulus).  

In symbols, given base b, exponent e, and modulus m, the modular exponentiation c is:  
c = be mod m. From the definition of c, it follows that 0 ≤ c < m.

For example, given b = 5, e = 3 and m = 13, the solution c = 8 is the remainder of dividing 53 = 125 by 13.

In [200]:
s = z3.Solver()

b, c, m = z3.Ints('b c m')
e = z3.Real('e')

s.add([ z3.And(0<=c, c<m) ])
#f = z3.Function('f',  z3.IntSort(), z3.IntSort(), z3.IntSort())
#s.add(f)
#print(s.sexpr())
s.add([b = 5, m = 13, c = 8])
z3.solve(((b**e) % m) == c)

s.check()
m = s.model()
m


SyntaxError: invalid syntax (<ipython-input-200-7a0b6431176b>, line 10)